In [1]:
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import re
import math
import keras
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn import set_config
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import FunctionTransformer
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LayerNormalization
from keras.layers import Dropout
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from pandas.api.types import is_numeric_dtype
from itertools import product
from joblib import dump
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer


In [2]:
df = pd.read_csv('/home/philippe/Documents/python project/EZStacking-main/dataset/iris.csv')

In [3]:
target_col = 'variety'

In [4]:
y = df[target_col]

In [5]:
X = df.drop(target_col, axis=1)

In [6]:
nb_features = len(X.columns.tolist())

In [7]:
nb_targets = len(y.unique())

In [8]:
layer_size = nb_features + nb_targets + 2

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [10]:
target_encoder = LabelEncoder() 
y_train = pd.Series(target_encoder.fit_transform(y_train)) 
y_test = pd.Series(target_encoder.transform(y_test))

In [11]:
num_ntree_processor = make_pipeline(SimpleImputer(strategy='mean', add_indicator=True), StandardScaler())

In [12]:
def K_Class(): 
    keras.backend.clear_session() 
#   neural network architecture: start 
    model = Sequential() 
    model.add(Dense(10 * layer_size, activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(Dropout(0.5)) 
#    model.add(LayerNormalization()) 
    model.add(Dense(layer_size, activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(Dropout(0.5)) 
#    model.add(LayerNormalization()) 
    model.add(Dense(nb_targets, activation='softmax')) 
#   neural network architecture: end   
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [13]:
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=20)

In [14]:
K_C = KerasClassifier(K_Class, batch_size=64, epochs=2000, callbacks=[es], validation_split=0.1, verbose=1) 
K_C._estimator_type = 'classifier'

In [15]:
level_0 = [ 
          ('LOGRL', make_pipeline(num_ntree_processor, LogisticRegression(solver='lbfgs', penalty='l2'))), 
          ('KERC', make_pipeline(num_ntree_processor, K_C)), 
          ]

In [16]:
level_1 = LogisticRegression()

In [17]:
model = StackingClassifier(level_0, final_estimator=level_1)

In [18]:
%%time 
set_config(display='diagram') 
model.fit(X_train, y_train)

Epoch 1/2000
2/2 [==============================] - 3s 615ms/step - loss: 1.7211 - accuracy: 0.2700 - val_loss: 1.1100 - val_accuracy: 0.0833
Epoch 2/2000
2/2 [==============================] - 0s 59ms/step - loss: 1.4274 - accuracy: 0.4100 - val_loss: 1.1035 - val_accuracy: 0.0833
Epoch 3/2000
2/2 [==============================] - 0s 76ms/step - loss: 1.2044 - accuracy: 0.4800 - val_loss: 1.0985 - val_accuracy: 0.0833
Epoch 4/2000
2/2 [==============================] - 0s 89ms/step - loss: 1.4106 - accuracy: 0.4600 - val_loss: 1.0925 - val_accuracy: 0.3333
Epoch 5/2000
2/2 [==============================] - 0s 63ms/step - loss: 1.4071 - accuracy: 0.3800 - val_loss: 1.0843 - val_accuracy: 0.5000
Epoch 6/2000
2/2 [==============================] - 0s 71ms/step - loss: 1.1739 - accuracy: 0.5300 - val_loss: 1.0758 - val_accuracy: 0.5000
Epoch 7/2000
2/2 [==============================] - 0s 74ms/step - loss: 1.0967 - accuracy: 0.5000 - val_loss: 1.0679 - val_accuracy: 0.5833
Epoch 8/2000

Epoch 59/2000
2/2 [==============================] - 0s 56ms/step - loss: 0.4091 - accuracy: 0.8800 - val_loss: 0.6126 - val_accuracy: 0.8333
Epoch 60/2000
2/2 [==============================] - 0s 70ms/step - loss: 0.4048 - accuracy: 0.8500 - val_loss: 0.6073 - val_accuracy: 0.8333
Epoch 61/2000
2/2 [==============================] - 0s 69ms/step - loss: 0.3333 - accuracy: 0.9000 - val_loss: 0.6018 - val_accuracy: 0.8333
Epoch 62/2000
2/2 [==============================] - 0s 74ms/step - loss: 0.3945 - accuracy: 0.8300 - val_loss: 0.5963 - val_accuracy: 0.8333
Epoch 63/2000
2/2 [==============================] - 0s 49ms/step - loss: 0.4133 - accuracy: 0.8500 - val_loss: 0.5902 - val_accuracy: 0.8333
Epoch 64/2000
2/2 [==============================] - 0s 50ms/step - loss: 0.3870 - accuracy: 0.8400 - val_loss: 0.5842 - val_accuracy: 0.8333
Epoch 65/2000
2/2 [==============================] - 0s 54ms/step - loss: 0.3731 - accuracy: 0.8600 - val_loss: 0.5776 - val_accuracy: 0.8333
Epoch 

2/2 [==============================] - 0s 92ms/step - loss: 0.2534 - accuracy: 0.8900 - val_loss: 0.3533 - val_accuracy: 1.0000
Epoch 117/2000
2/2 [==============================] - 0s 96ms/step - loss: 0.2502 - accuracy: 0.9200 - val_loss: 0.3504 - val_accuracy: 1.0000
Epoch 118/2000
2/2 [==============================] - 0s 52ms/step - loss: 0.2523 - accuracy: 0.9200 - val_loss: 0.3477 - val_accuracy: 1.0000
Epoch 119/2000
2/2 [==============================] - 0s 65ms/step - loss: 0.2605 - accuracy: 0.9500 - val_loss: 0.3445 - val_accuracy: 1.0000
Epoch 120/2000
2/2 [==============================] - 0s 108ms/step - loss: 0.2330 - accuracy: 0.9400 - val_loss: 0.3412 - val_accuracy: 1.0000
Epoch 121/2000
2/2 [==============================] - 0s 64ms/step - loss: 0.2209 - accuracy: 0.9200 - val_loss: 0.3374 - val_accuracy: 1.0000
Epoch 122/2000
2/2 [==============================] - 0s 69ms/step - loss: 0.2164 - accuracy: 0.9700 - val_loss: 0.3330 - val_accuracy: 1.0000
Epoch 123/200

2/2 [==============================] - 0s 43ms/step - loss: 0.2212 - accuracy: 0.9100 - val_loss: 0.2013 - val_accuracy: 1.0000
Epoch 174/2000
2/2 [==============================] - 0s 39ms/step - loss: 0.1681 - accuracy: 0.9500 - val_loss: 0.2000 - val_accuracy: 1.0000
Epoch 175/2000
2/2 [==============================] - 0s 38ms/step - loss: 0.1544 - accuracy: 0.9600 - val_loss: 0.1990 - val_accuracy: 1.0000
Epoch 176/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.1666 - accuracy: 0.9700 - val_loss: 0.1988 - val_accuracy: 1.0000
Epoch 177/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.1707 - accuracy: 0.9600 - val_loss: 0.1986 - val_accuracy: 1.0000
Epoch 178/2000
2/2 [==============================] - 0s 65ms/step - loss: 0.1479 - accuracy: 0.9700 - val_loss: 0.1985 - val_accuracy: 1.0000
Epoch 179/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.2334 - accuracy: 0.9500 - val_loss: 0.1981 - val_accuracy: 1.0000
Epoch 180/2000

2/2 [==============================] - 0s 85ms/step - loss: 0.1138 - accuracy: 0.9600 - val_loss: 0.1613 - val_accuracy: 1.0000
Epoch 231/2000
2/2 [==============================] - 0s 104ms/step - loss: 0.1436 - accuracy: 0.9400 - val_loss: 0.1618 - val_accuracy: 1.0000
Epoch 232/2000
2/2 [==============================] - 0s 59ms/step - loss: 0.1481 - accuracy: 0.9600 - val_loss: 0.1613 - val_accuracy: 1.0000
Epoch 233/2000
2/2 [==============================] - 0s 44ms/step - loss: 0.1472 - accuracy: 0.9500 - val_loss: 0.1593 - val_accuracy: 1.0000
Epoch 234/2000
2/2 [==============================] - 0s 62ms/step - loss: 0.1398 - accuracy: 0.9700 - val_loss: 0.1565 - val_accuracy: 1.0000
Epoch 235/2000
2/2 [==============================] - 0s 61ms/step - loss: 0.1230 - accuracy: 0.9700 - val_loss: 0.1531 - val_accuracy: 1.0000
Epoch 236/2000
2/2 [==============================] - 0s 93ms/step - loss: 0.1382 - accuracy: 0.9600 - val_loss: 0.1503 - val_accuracy: 1.0000
Epoch 237/200

...

Epoch 108/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.2777 - accuracy: 0.9506 - val_loss: 0.5619 - val_accuracy: 0.8889
Epoch 109/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.3114 - accuracy: 0.8642 - val_loss: 0.5555 - val_accuracy: 0.8889
Epoch 110/2000
2/2 [==============================] - 0s 37ms/step - loss: 0.2905 - accuracy: 0.9506 - val_loss: 0.5500 - val_accuracy: 0.8889
Epoch 111/2000
2/2 [==============================] - 0s 45ms/step - loss: 0.3682 - accuracy: 0.9012 - val_loss: 0.5451 - val_accuracy: 0.8889
Epoch 112/2000
2/2 [==============================] - 0s 45ms/step - loss: 0.3510 - accuracy: 0.9136 - val_loss: 0.5401 - val_accuracy: 0.8889
Epoch 113/2000
2/2 [==============================] - 0s 39ms/step - loss: 0.2631 - accuracy: 0.9259 - val_loss: 0.5331 - val_accuracy: 0.8889
Epoch 114/2000
2/2 [==============================] - 0s 40ms/step - loss: 0.2902 - accuracy: 0.9259 - val_loss: 0.5265 - val_accuracy: 0.8889

2/2 [==============================] - 0s 32ms/step - loss: 0.2510 - accuracy: 0.9259 - val_loss: 0.2918 - val_accuracy: 1.0000
Epoch 166/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.2916 - accuracy: 0.8642 - val_loss: 0.2888 - val_accuracy: 1.0000
Epoch 167/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.2578 - accuracy: 0.9506 - val_loss: 0.2859 - val_accuracy: 1.0000
Epoch 168/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.1936 - accuracy: 0.9506 - val_loss: 0.2825 - val_accuracy: 1.0000
Epoch 169/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.1864 - accuracy: 0.9506 - val_loss: 0.2781 - val_accuracy: 1.0000
Epoch 170/2000
2/2 [==============================] - 0s 37ms/step - loss: 0.1499 - accuracy: 0.9630 - val_loss: 0.2747 - val_accuracy: 1.0000
Epoch 171/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.2431 - accuracy: 0.9383 - val_loss: 0.2707 - val_accuracy: 1.0000
Epoch 172/2000

2/2 [==============================] - 0s 35ms/step - loss: 0.1575 - accuracy: 0.9630 - val_loss: 0.1671 - val_accuracy: 1.0000
Epoch 223/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.1679 - accuracy: 0.9506 - val_loss: 0.1659 - val_accuracy: 1.0000
Epoch 224/2000
2/2 [==============================] - 0s 40ms/step - loss: 0.2142 - accuracy: 0.9012 - val_loss: 0.1644 - val_accuracy: 1.0000
Epoch 225/2000
2/2 [==============================] - 0s 39ms/step - loss: 0.1655 - accuracy: 0.9753 - val_loss: 0.1627 - val_accuracy: 1.0000
Epoch 226/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.1914 - accuracy: 0.9506 - val_loss: 0.1608 - val_accuracy: 1.0000
Epoch 227/2000
2/2 [==============================] - 0s 38ms/step - loss: 0.1690 - accuracy: 0.9630 - val_loss: 0.1589 - val_accuracy: 1.0000
Epoch 228/2000
2/2 [==============================] - 0s 48ms/step - loss: 0.1375 - accuracy: 0.9753 - val_loss: 0.1567 - val_accuracy: 1.0000
Epoch 229/2000

2/2 [==============================] - 0s 34ms/step - loss: 0.1988 - accuracy: 0.9136 - val_loss: 0.1029 - val_accuracy: 1.0000
Epoch 280/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.1707 - accuracy: 0.9506 - val_loss: 0.1022 - val_accuracy: 1.0000
Epoch 281/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.1501 - accuracy: 0.9506 - val_loss: 0.1019 - val_accuracy: 1.0000
Epoch 282/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0861 - accuracy: 1.0000 - val_loss: 0.1024 - val_accuracy: 1.0000
Epoch 283/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.1237 - accuracy: 0.9753 - val_loss: 0.1030 - val_accuracy: 1.0000
Epoch 284/2000
2/2 [==============================] - 0s 31ms/step - loss: 0.1490 - accuracy: 0.9506 - val_loss: 0.1030 - val_accuracy: 1.0000
Epoch 285/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0960 - accuracy: 0.9753 - val_loss: 0.1031 - val_accuracy: 1.0000
Epoch 286/2000

StackingClassifier(estimators=[('LOGRL',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('simpleimputer',
                                                                  SimpleImputer(add_indicator=True)),
                                                                 ('standardscaler',
                                                                  StandardScaler())])),
                                                ('logisticregression',
                                                 LogisticRegression())])),
                               ('KERC',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('simpleimputer',
                                                                  SimpleImputer(add_indicator=True)),
                                                                 ('standardscaler',
                                                                  StandardScaler())])),
                                                ('kerasclassifier',
                                                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f80093f9df0>)]))],
                   final_estimator=LogisticRegression())

In [19]:
model.score(X_test, y_test)

2/2 [==============================] - 0s 6ms/step


0.9473684210526315